<a href="https://colab.research.google.com/github/jbarrasa/goingmeta/blob/main/session18/Easy_full_graph_migration_from_triple_stores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Migration from Stardog

In [ ]:
pip install pystardog neo4j

STEP1: Connect to the Stardog instance using the pystardog driver

In [5]:
import stardog
from neo4j import GraphDatabase, basic_auth

In [6]:
stardog_connection_details = {
  'endpoint': 'https://your-instance-id.stardog.cloud:5820',
  'username': 'your_user',
  'password': 'your_pwd'
}

In [7]:
conn = stardog.Connection('test-database', **stardog_connection_details)

OPTIONAL: Populate the DB if empty. Not needed if your DB is already pre-loaded

In [ ]:
conn.begin()
conn.add(stardog.content.URL('https://github.com/stardog-union/stardog-tutorials/raw/master/music/music.ttl.gz'))
conn.commit()

STEP2: Connect to Neo4j and initialise the DB

In [9]:
neo_db = GraphDatabase.driver(
  "bolt://your.neo4j.instance:7687",
  auth=basic_auth("your-user", "your-pwd"), database="neo4j")

neo_db.execute_query('CREATE CONSTRAINT n10s_unique_uri IF NOT EXISTS FOR (r:Resource) REQUIRE r.uri IS UNIQUE')
neo_db.execute_query('call n10s.graphconfig.init()')

STEP3: Run the migration!

We will use
* the method `conn.export()` to export the whole graph from Stardog
* the `n10s.rdf.import.inline` procedure to import the RDF into Neo4j


In [ ]:
neo_db.execute_query('call n10s.rdf.import.inline($payload,"Turtle")', {"payload": conn.export().decode('utf-8')})

# and close the connection
neo_db.close()

# Migration from Ontotext

STEP1: Run SPARQL construct query returning all triples from the DB

In [ ]:
import requests
import urllib.parse

endpoint = "https://i40kg.ontotext.com/graphdb/repositories/i40kg"
sparql = " construct { ?s ?p ?o } WHERE { ?s ?p ?o} "
export_result = requests.get(endpoint, params = {"query": sparql , "Accept" : "text/turtle"})


STEP2: Connect to Neo4j and initialise the DB (identical to previous example)

In [ ]:
neo_db = GraphDatabase.driver(
  "bolt://your.neo4j.instance:7687",
  auth=basic_auth("your-user", "your-pwd"),database="i40kg")

neo_db.execute_query('CREATE CONSTRAINT n10s_unique_uri IF NOT EXISTS FOR (r:Resource) REQUIRE r.uri IS UNIQUE')
neo_db.execute_query('call n10s.graphconfig.init()')

STEP3: Run the migration!

Same as previous example but using the result of the previous request

In [ ]:
if export_result.status_code == 200:
  neo_db.execute_query('call n10s.rdf.import.inline($payload,"Turtle")', {"payload": export_result.text })

# and close the connection
neo_db.close()